In [ ]:
import os
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter data and create a table to retrieve rows where total_votes count is = or > 20
vine_20_df = vine_df.filter("total_votes>=20")
vine_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [ ]:
# Filter and create a table that helpful_votes divided by total_votes is equal to or greater than 50%.
vine_50_df = vine_20_df.filter("helpful_votes/total_votes >= 0.5")
vine_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [ ]:
# The review was written as part of the Vine program (paid).
vine_paid_df = vine_50_df.filter(vine_50_df.vine == "Y")
vine_paid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|
| R5QTOT1GVR8RG|          4|           16|         20|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# The review was not part of the Vine program (unpaid).
vine_unpaid_df = vine_50_df.filter(vine_50_df.vine == "N")
vine_unpaid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
total_reviews = vine_paid_df.select("star_rating").count()

In [ ]:
total_reviews_un = vine_unpaid_df.select("star_rating").count()

In [ ]:
five_stars = vine_paid_df.filter("star_rating=5").select("star_rating").count()

In [ ]:
five_stars_un = vine_unpaid_df.filter("star_rating=5").select("star_rating").count()

In [ ]:
print("Paid")
print(f"Total number of reviews: {total_reviews}")
print(f"The number of 5-star: {five_stars}")
print(f"The percentage of 5-star reviews: {five_stars/total_reviews*100}%")

Paid
Total number of reviews: 49
The number of 5-star: 9
The percentage of 5-star reviews: 18.367346938775512%


In [ ]:
print("Unpaid")
print(f"Total number of reviews: {total_reviews_un}")
print(f"The number of 5-star: {five_stars_un}")
print(f"The percentage of 5-star reviews: {five_stars_un/total_reviews_un*100}%")

Unpaid
Total number of reviews: 151400
The number of 5-star: 78061
The percentage of 5-star reviews: 51.55944517833554%
